In [25]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Load the data
from sklearn.datasets import load_breast_cancer
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [26]:
!wget https: // raw.githubusercontent.com/datasets/breast-cancer/master/data/breast-cancer.csv

Missing host/domain in URI 'https:'
Failed to parse URI 'https:'
Failed to parse URI '//'
HSTS in effect for raw.githubusercontent.com:80


7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://raw.githubusercontent.]87Saving 'breast-cancer.csv.3'
87breast-cancer.csv.3  100% [=============================>]    1.64K    --.-KB/s87HTTP response 200  [https://raw.githubusercontent.com/datasets/breast-cancer/master/data/breast-cancer.csv]
87breast-cancer.csv.3  100% [=============================>]    1.64K    --.-KB/s87[Files: 1  Bytes: 1.64K [1.85KB]8

In [27]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [28]:
data = load_breast_cancer()

X = data.data[:, :-1]
y = data.data[:, -1]
y = y.reshape((-1, 1))


In [29]:

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Классы нейронки и слоев

In [30]:

def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

def compute_cost(y_pred, y_true):
    m = y_true.shape[0]
    cost = -(1 / m) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return cost


class Layer:
    def __init__(self, input_shape: int, output_shape: int):
        self.weights = np.random.randn(input_shape, output_shape)
        self.bias = np.zeros((1, output_shape))


class NeuralNetwork:
    def __init__(self, layers, activation):
        self.layers = layers
        self.activation = activation

    def predict(self, X):
        return self.forward(X)[-1]

    def forward(self, X):
        a_hist = [X]
        for l in self.layers:
            z = np.dot(a_hist[-1], l.weights) + l.bias
            a = self.activation(z)
            a_hist.append(a)
        return a_hist

    def backward(self, y_true, learning_rate, a_hist):
        m = y_true.shape[0]
        dz = a_hist[-1] - y_true
        d_hist = []
        prev = None
        for l, a in zip(self.layers[::-1], a_hist[0:-1:-1]):
            if prev:
                lp, ap = prev 
                dz = np.dot(dz, lp.weights.T) * ap * (1 - ap)
            dw = (1 / m) * np.dot(a.T, dz)
            db = (1 / m) * np.sum(dz, axis=0, keepdims=True)
            d_hist.append((dw, db))
            prev = l, a

        for l, (dw, db) in zip(self.layers[::-1], d_hist):
            l.weights = l.weights - learning_rate * dw
            l.bias = l.bias - learning_rate * db

    def train(self, X, y_true, learning_rate, num_iterations):
        for i in tqdm(range(num_iterations)):
            a_hist = self.forward(X)
            cost = compute_cost(a_hist[-1], y_true)
            self.backward(y_true, learning_rate, a_hist)
            if i % 1000 == 0:
                print(f"Cost after iteration {i}: {cost:.4f}")


In [31]:
nn = NeuralNetwork(
    [
        Layer(X_train.shape[1], 15),
        Layer(15, 10),
        Layer(10, 1)
    ], sigmoid)
print(X_train.shape)
print(y_train.shape)
nn.train(X_train, y_train, 0.1, 10000)

(455, 29)
(455, 1)


/tmp/ipykernel_24097/3211219221.py:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(num_iterations)):


  0%|          | 0/10000 [00:00<?, ?it/s]

Cost after iteration 0: 0.3782
Cost after iteration 1000: 0.3782
Cost after iteration 2000: 0.3782
Cost after iteration 3000: 0.3782
Cost after iteration 4000: 0.3782
Cost after iteration 5000: 0.3782
Cost after iteration 6000: 0.3782
Cost after iteration 7000: 0.3782
Cost after iteration 8000: 0.3782
Cost after iteration 9000: 0.3782


In [32]:
# Evaluate the neural network on the test set
y_pred = nn.predict(X_test)

y_pred_e = np.round(y_pred, 2)
y_test_e = np.round(y_test, 2)
print(y_pred_e)
print(y_test_e)
accuracy = np.mean(y_pred_e == y_test_e)
print(f"Test accuracy: {accuracy}")

[[0.24]
 [0.2 ]
 [0.27]
 [0.35]
 [0.32]
 [0.39]
 [0.26]
 [0.24]
 [0.35]
 [0.16]
 [0.1 ]
 [0.14]
 [0.1 ]
 [0.42]
 [0.17]
 [0.56]
 [0.18]
 [0.31]
 [0.36]
 [0.25]
 [0.3 ]
 [0.24]
 [0.36]
 [0.12]
 [0.09]
 [0.41]
 [0.17]
 [0.26]
 [0.11]
 [0.41]
 [0.2 ]
 [0.17]
 [0.48]
 [0.15]
 [0.24]
 [0.19]
 [0.09]
 [0.2 ]
 [0.11]
 [0.08]
 [0.23]
 [0.28]
 [0.17]
 [0.13]
 [0.47]
 [0.18]
 [0.37]
 [0.15]
 [0.11]
 [0.18]
 [0.08]
 [0.24]
 [0.32]
 [0.47]
 [0.39]
 [0.14]
 [0.4 ]
 [0.41]
 [0.25]
 [0.23]
 [0.1 ]
 [0.24]
 [0.26]
 [0.1 ]
 [0.11]
 [0.09]
 [0.38]
 [0.49]
 [0.16]
 [0.27]
 [0.46]
 [0.27]
 [0.1 ]
 [0.51]
 [0.34]
 [0.26]
 [0.13]
 [0.23]
 [0.14]
 [0.31]
 [0.21]
 [0.14]
 [0.21]
 [0.2 ]
 [0.09]
 [0.4 ]
 [0.4 ]
 [0.4 ]
 [0.28]
 [0.22]
 [0.14]
 [0.14]
 [0.33]
 [0.19]
 [0.23]
 [0.31]
 [0.22]
 [0.36]
 [0.22]
 [0.25]
 [0.07]
 [0.23]
 [0.34]
 [0.3 ]
 [0.11]
 [0.19]
 [0.37]
 [0.31]
 [0.18]
 [0.09]
 [0.15]
 [0.18]
 [0.42]
 [0.19]]
[[0.09]
 [0.07]
 [0.08]
 [0.09]
 [0.08]
 [0.12]
 [0.09]
 [0.08]
 [0.1 ]
 [0.08]
 [0.07]